# Creating Historical Database

In [1]:
import os
import tqdm
import zipfile
import numpy as np
import pandas as pd

# speeds up pandas
# reference: https://towardsdatascience.com/one-word-of-code-to-stop-using-pandas-so-slowly-793e0a81343c
import swifter

from collections import defaultdict

In [2]:
datapath = './data'
unzip_path = './unzipped_data'

<br>

## Getting layout info

In [3]:
header = pd.read_csv('layout_header.csv', sep=';', encoding='utf-8-sig', index_col=0)
layout = pd.read_csv('layout_table.csv', sep=';', encoding='utf-8-sig', index_col=0)

### preparing info columns

In [4]:
layout['PREFIX'] = layout['NAME'].apply(lambda x:str(x).split(' -')[0])  # short name for each column

<br>

## Importing historical data

### unzipping files

In [5]:
zipNames = os.listdir(datapath)

In [6]:
for file in zipNames:
    with zipfile.ZipFile(f'./data/{file}', 'r') as zip_ref:
        zip_ref.extractall('./unzipped_data')

### Importing into pandas dataframe

In [7]:
unzipNames = os.listdir(unzip_path)

In [8]:
df = pd.read_table(f'./unzipped_data/{unzipNames[0]}')

In [11]:
df.head()

,00COTAHIST.2010BOVESPA 20101230
0,012010010402ABCB4 010ABC BRASIL PN EJ ...
1,012010010496ABCB4F 020ABC BRASIL PN EJ ...
2,012010010402ABNB3 010ABNOTE ON ED ...
3,012010010496ABNB3F 020ABNOTE ON ED ...
4,012010010462ABNB3T 030ABNOTE ON ED ...


## parsing layout to database

### header

In [14]:
h = df.columns[0].strip()  # header string (column name of df)

head = defaultdict()
for row in range(header.shape[0]):
    init = header.loc[row, 'INIT'] - 1  # start position
    end = header.loc[row, 'END']        # end position (p.s: Pythonic way of indexing ;)
    
    field = header.loc[row,'NAME']      # field name
    
    head[field] = h[init:end].strip()   # values (also removing spaces)

head = dict(head)                       # changing from defaultdict to regular dict

### data

In [13]:
# counter = 0              # counts row number
# data = pd.DataFrame()    
# for i in range(df.shape[0]):
#     val = df.loc[i].values[0]
    
#     for row in range(layout.shape[0]):
#         init = layout.loc[row, 'INIT'] - 1        # start position
#         end = layout.loc[row, 'END']              # end position (p.s: Pythonic way of indexing ;)
#         field = layout.loc[row,'PREFIX']          # field name

#         data.loc[counter,field] = val[init:end]   # values into row = counter
    
#     counter += 1  # advancing counter
        

In [152]:
def get_values(string_row, layout, prefix):
    '''
    applies layout to a row of historical data table.
    
    Arguments:
        - string_row (str): row of historical dataframe (just read the historical data 
            text file into pandas dataframe with pandas.readtable and parse one row at a time)
        - prefix (str): prefix of the column/field name
        - layout (pandas.dataframe): layout table
    
    returns the value of that field for the parsed row.
    '''
    field_info = layout[layout['PREFIX'] == prefix]
    
    init = field_info.loc[:,'INIT'].values[0] - 1        # start position
    end = field_info.loc[:,'END'].values[0]              # end position (p.s: Pythonic way of indexing ;)
    
    return string_row[init:end]

def get_multiple_values(string_row, layout, prefixes="all"):
    '''
    Gets multiple values from a row of historical data table (according to layout).
    
    Arguments:
        - string_row (str): row of historical dataframe (just read the historical data 
            text file into pandas dataframe with pandas.readtable and parse one row at a time)
        - prefixes (str): prefix of the column/field name
        - layout (pandas.dataframe): layout table
    
    returns the value of that field for the parsed row.
    '''
    if prefixes == "all":
        prefixes = layout["PREFIX"]
        
    values = []
    for prefix in prefixes:
        values.append(get_values(string_row, layout, prefix))
    
    return values

In [126]:
old_col = df.columns[0]
for p in tqdm.tqdm(layout['PREFIX']):
    df[p] = df[old_col].apply(lambda x: get_values(x, layout, p))


100%|██████████| 26/26 [1:49:48<00:00, 253.41s/it]


In [154]:
layout

,NAME,CONTENT,KIND AND SIZE,INIT,END,PREFIX
0,TIPREG - TIPO DE REGISTRO,FIXO “01”,N(02),1,2,TIPREG
1,DATA DO PREGÃO,FORMATO “AAAAMMDD”,N(08),3,10,DATA DO PREGÃO
2,CODBDI - CÓDIGO BDI,UTILIZADO PARA CLASSIFICAR OS PAPÉIS NA EMISSÃ...,X(02),11,12,CODBDI
3,CODNEG - CÓDIGO DE NEGOCIAÇÃO DO PAPEL,NaN,X(12),13,24,CODNEG
4,TPMERC - TIPO DE MERCADO,CÓD. DO MERCADO EM QUE O PAPEL ESTÁ CADASTRADO...,N(03),25,27,TPMERC
5,NOMRES - NOME RESUMIDO DA EMPRESA EMISSORA DO ...,NaN,X(12),28,39,NOMRES
6,ESPECI - ESPECIFICAÇÃO DO PAPEL,VER TABELA ANEXA,X(10),40,49,ESPECI
7,PRAZOT - PRAZO EM DIAS DO MERCADO A TERMO,NaN,X(03),50,52,PRAZOT
8,MODREF - MOEDA DE REFERÊNCIA,MOEDA USADA NA DATA DO PREGÃO,X(04),53,56,MODREF
9,PREABE - PREÇO DE ABERTURA DO PAPEL-MERCADO NO...,NaN,(11)V99,57,69,PREABE


### exporting data

In [151]:
# metadata
file_name = unzipNames[0]
file_name = file_name.split('.')[0]

with open(f'./output/{file_name}-metadata.txt','w') as f:
    f.write(str(head))

df.to_csv(f'./output/{file_name}.csv', sep=';', encoding='utf-8-sig')

**end**